In [1]:
# projeto_transacoes_bancarias -- DS-PY-17 - logica de programacao II
# readme link here: 
# https://github.com/allansuzuki/ADA_classes/blob/main/DS-PY-Data-Science/DS-PY-017%20L%C3%93GICA%20DE%20PROGRAMA%C3%87%C3%83O%20II%20(PY)/Material%20do%20Aluno/projeto_README.md
# 
# Esse programa é um sistema de gestao de transacoes de uma conta bancária pessoal
# no qual os dados são de transações e possuem seu valor, a categoria do gasto e seu ID.
# 
# Teu objetivo é completar esse sistema CRUD (Create-Read-Update-Delete) simples 
# para ver dados de transacao da tua conta pessoal, criar, editar e excluir transações.
# Também deve fazer com que o programa NUNCA pare, ou seja,
# caso ocorra um possível erro, deve validar as entradas, detectar erros e avisar o usuário
# mas o programa não deve parar.
#
#
# Notas importantes: 
# 1. As funções que geram os dados e criam a interface do sistema já estão prontas. 
# por favor não as altere.
#
# 2. Depois das funções do sistema estão as funções do programa
# No qual podem alterar à vontade, exceto o nome das funções
# Ou seja, podem criar funções, adicionar ou remover parâmetros, 
# mas não alterar o nome das funções existentes.
#
# 3. Coloque opções de navegabilidade em cada janela que o usuário estiver.
# Por exemplo, se ele escolher a opcao "alterar transacao" sem querer, tem que ter a opcao de voltar para a tela anterior ou inicial.
#
# 4. Caso por qualquer motivo queira os dados originais novamente,
# apage o json `transactions` na pasta `data` e inicie o programa novamente para gerar os dados.
# Os valores serão os mesmos, porém os UUID NÃO serão os mesmos!!
#
# Critérios (pontos):
#   tarefas validacoes  total
# C     10      15       25
# R     25      25       50
# U     10      10       20
# D     2.5     2.5      5
#
#
# Boa sorte e divirtam-se :)
# ------------------------------------------------------------------------------

In [82]:
# -----------------------
# depencies
# -----------------------
import json
import os
import uuid
import random
import sys
from IPython.display import clear_output

# -----------------------
# load settings
# -----------------------
sys.path.append('./data/')
from data import settings

# -----------------------
# SYSTEM functions 
# -----------------------
# não alterar nada das funções de system
def criar_transacoes(num_transacoes, proporcao_categorias, seed=settings.seed):
    assert sum([proporcao_categorias[k] for k in proporcao_categorias])==1, '`proporcao_categorias` não soma 100%! Favor rever.'

    # garantir reprodutibilidade dos valores
    random.seed(seed)

    # Calcula o número de transações por categoria com base na proporção
    numero_transacoes_por_categoria = {categoria: int(num_transacoes * proporcao) for categoria, proporcao in proporcao_categorias.items()}
    
    transacoes = []
    
    # Gera as transações
    for categoria, quantidade in numero_transacoes_por_categoria.items():
        for _ in range(quantidade):
            transacao = {
                "UUID": str(uuid.uuid4()),
                "valor": round(random.uniform(1.0, 1000.0), 2),  # Preço aleatório entre 1 e 1000
                "categoria": categoria
            }
            transacoes.append(transacao)
    
    return transacoes

def salvar_json(transacoes, path2save, filename):
    # create path if not exist
    if not os.path.exists(path2save):
        os.makedirs(path2save)
    with open(os.path.join(path2save,filename), "w") as file:
        json.dump(transacoes, file, indent=4)
    print(f"Arquivo salvo em: {os.path.abspath(os.path.curdir)+'/'+path2save+'/'+filename}")

def criar_bd(num_transacoes:int = 10000, proporcao_categorias:list = settings.categorias_proporcao, path2save="./data", filename='transactions.json'):
    salvar_json(criar_transacoes(num_transacoes,  proporcao_categorias),
                path2save, filename
    )

def load_bd(filepath='./data/transactions.json'):
    with open(filepath, "r") as file:
        bd = json.load(file)
    return bd

def tela_inicial(usuario, conta):
    print(f"Bem-vindo, {usuario}!")
    print(f"Conta: {conta}")
    print("\nEste programa permite gerenciar transações de sua conta pessoal.")
    print("\nEscolha uma das opções abaixo:")
    print("1. Visualizar relatórios")
    print("2. Cadastrar transações")
    print("3. Editar transações")
    print("4. Excluir transações")
    print("-" * 10)
    print("0. Sair")
    print('\n')

In [83]:
def validar_opcao_escolhida():
    while True:
        confirmacao = input("Deseja continuar? \n1: Sim\n2: Não").strip()
        if confirmacao == "1" or confirmacao == "2":
            clear_output(wait = True)
            break
        else:
            print("Opção inválida. ** Digite 1 para Sim e 2 para Não **")
            clear_output(wait = True)
    return confirmacao


def validar_valor(valor):
    try:
        valor = float(valor)
        if valor == 0:
            raise Exception("O valor não pode ser zero.")
        elif valor < 0:
            raise Exception("O valor não pode ser negativo.")
        clear_output(wait = True)
        return valor
    except ValueError:
        print("O valor inserido não é válido. ** Use apenas números e ponto como separador decimal **")
        clear_output(wait = True)
    except Exception as erro:
        print(erro)
        clear_output(wait = True)
    except:
        print("Valor inválido.")
        clear_output(wait = True)
    return None


def validar_categoria(categoria):
    dict_categoria = {
        "1": "casa",
        "2": "lazer",
        "3": "viagens",
        "4": "investimentos",
        "5": "transferencias",
        "6": "saude",
        "7": "alimentacao"
        }
    try:
        categoria = dict_categoria[categoria]
        clear_output(wait = True)
        return categoria
    except KeyError:
        print("A categoria não existe. ** Digite um número entre 1 e 7 **")
        clear_output(wait = True)
    except:
        print("Opção inválida.")
        clear_output(wait = True)
    return None


def repetir_operacao(operacao):
    while True:
        repetir_operacao = input(f"Deseja {operacao} outra transação? \n1: Sim\n2: Não").strip()
        if repetir_operacao == "1" or repetir_operacao == "2":
            break 
        else:
            print("Opção inválida. ** Digite 1 para Sim e 2 para Não **")
            clear_output(wait = True)
    return repetir_operacao


def cadastrar_transacao(valor, categoria):
    transacoes = load_bd()
    transacao = {
        "UUID": str(uuid.uuid4()),
        "valor": valor,
        "categoria": categoria
    }
    transacoes.append(transacao)
    return salvar_json(transacoes, "./data", "transactions.json")
    

def run():
    """
    Esta é a função principal que vai rodar o programa
    """  
    usuario = input('Olá! Vamos iniciar seu atendimento. Por favor, insira seu nome:').title().strip()
    conta = input('Insira o número da sua conta:').strip()

    opcao_escolhida = None

    while True:

        # exibe a tela inicial
        tela_inicial(usuario, conta)
        opcao_escolhida = input().strip()
        clear_output(wait = True)
        
        if opcao_escolhida == "0":
            print("SAIR")
            break

        elif opcao_escolhida == "1":
            # visualizar_relatorios()
            print("OPÇÃO 1")

        elif opcao_escolhida == "2":
            print("*** CADASTRAR TRANSAÇÃO ***")
            confirmacao = validar_opcao_escolhida()
            if confirmacao == "2":
                clear_output()
                continue
            while True:
                while True:
                    valor = input("Informe o valor da transação: ").strip()
                    valor_validado = validar_valor(valor)
                    if valor_validado is not None:
                        break
                while True:    
                    categoria = input("Escolha a categoria da transação: \n1: casa\n2: lazer\n3: viagens\n4: investimentos\n5: transferencias\n6: saude\n7: alimentacao").strip()
                    categoria_validada = validar_categoria(categoria)
                    if categoria_validada is not None:
                        break
                print(f'Valor: R$ {valor_validado:.2f}')
                print(f'Categoria: {categoria_validada}')
                
                cadastrar_transacao(valor, categoria)
                print("Transação cadastrada com sucesso!")
                clear_output(wait = True)

                cadastrar_mais_transacoes = repetir_operacao("cadastrar")
                if cadastrar_mais_transacoes == "2":
                    break

        
        elif opcao_escolhida == "3":
            # editar_transacao_por_ID()
            print("OPÇÃO 3")
        
        elif opcao_escolhida == "4":
            # excluir_transacao()
            print("PÇÃO 4")
        
        else:
            print("Opção inválida, digite novamente.")

In [ ]:

# -----------------------
# MAIN SCRIPT
# -----------------------
if __name__ == "__main__":
    
    # criar o banco de dados caso ele não exista
    print(os.path.abspath('.'))
    if not os.path.exists('./data/transactions.json'):
        criar_bd()
    
    # load bd 
    bd = load_bd()
   
    # inicia o programa
    run()